-------------------------
### Author: Anthony Melin
### Date: 2019 August 14
-------------------------

# Import

In [13]:
from Socket.CameraSocket import CameraSocket
from Socket.RayCollisionSocket import RayCollisionSocket
from Socket.AnnotationSocket import AnnotationSocket

from Tensorflow.ObjectDetection import *
from Tensorflow.ObjectDetector import ObjectDetector

from Scenario.SimpleObjectDetection import SimpleObjectDetection
from Scenario.PenNearNotebook import PenNearNotebook

import cv2
import sys
import numpy as np

# Global variables
## Model files and classes

In [14]:
# MODEL_DIRECTORY = "MODEL_DIRECTORY"
MODEL_DIRECTORY = "C:\\Users\\Anthony\\Documents\\NATAssistanceAR\\ModelsForNAT\\model_3"

GRAPH = MODEL_DIRECTORY + "/frozen_inference_graph.pb"
LABELS = MODEL_DIRECTORY + "/labelmap.pbtxt"
NUM_CLASSES = 99

## Host and port

In [15]:
HOST = "192.168.1.21"

CAMERA_PORT = 9999
COLLISION_PORT = 9998
ANNOTATION_PORT = 9997

## Scenario

In [16]:
scenario = PenNearNotebook()

# Object detection configuration

### Load model

In [17]:
category_index = load_categories(LABELS, NUM_CLASSES)
sess, inputs, outputs = load_model(GRAPH)

### Set frame detector

In [18]:
detector = ObjectDetector(sess, inputs, outputs, category_index)
detector.SetThreshold(60)
detector.draw = True

# Define sockets and await connection

### Camera

Exit if connection failed

In [19]:
cameraSocket = CameraSocket().Bind(HOST, CAMERA_PORT)
connected = cameraSocket.WaitConnection()
if not connected:
    cameraSocket.close()
    sys.exit(0)

### Ray collision

Exit if connection failed

In [20]:
collisionSocket = RayCollisionSocket().Bind(HOST, COLLISION_PORT)
connected = collisionSocket.WaitConnection()
if not connected:
    cameraSocket.close()
    collisionSocket.close()
    sys.exit(0)

### Annotation

Exit if connection failed

In [21]:
annotationSocket = AnnotationSocket().Bind(HOST, ANNOTATION_PORT)
connected = annotationSocket.WaitConnection()
if not connected:
    cameraSocket.close()
    collisionSocket.close()
    annotationSocket.close()
    sys.exit(0)

# Loop

In [10]:
while True:

    # frame
    frame = cameraSocket.GetFrame()
    
    # detection
    detected = detector.Detect(frame)

    # ray collision test
    positions = collisionSocket.AskPositions(detected["centers"])
    
    # annotation
    if len(positions) > 0 and len(positions) == len(detected["centers"]):
        detected["positions"] = positions
        scenario.Update(detected, annotationSocket, frame=frame)
    
    # display
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('q'): break


# cameraSocket.Exit()
# collisionSocket.Exit()
# annotationSocket.Exit()

# cameraSocket.close()
# collisionSocket.close()
# annotationSocket.close()

# cv2.destroyAllWindows()

                              Lunchbox vec3(            9,            9,            9 )
Lunchbox vec3(          7.2,          7.2,          7.2 )
Lunchbox vec3(     -0.31931,    -0.383609,      1.62709 )
Lunchbox vec3(     -0.31931,    -0.383609,      1.62709 )
Lunchbox vec3(     -0.31931,    -0.383609,      1.62709 )
Lunchbox vec3(     -0.31931,    -0.383609,      1.62709 )
Lunchbox vec3(     -0.31931,    -0.383609,      1.62709 )
Lunchbox vec3(          7.2,          7.2,          7.2 )
                              Bottle vec3(            9,            9,            9 )
Lunchbox vec3(          7.2,          7.2,          7.2 )
Bottle vec3(          7.2,          7.2,          7.2 )
                              Notebook vec3(            9,            9,            9 )
Lunchbox vec3(          7.2,          7.2,          7.2 )
Bottle vec3(          7.2,          7.2,          7.2 )
Notebook vec3(          7.2,          7.2,          7.2 )
Lunchbox vec3(          7.2,          7.2,    

In [30]:
annotationSocket.Draw("update_text", -0.2,-1.6,2, "Pen", "green")
annotationSocket.Draw("update_text", 0,-1.5,2, "Notebook", "green")

In [12]:
cameraSocket.close()
collisionSocket.close()
annotationSocket.close()
cv2.destroyAllWindows()